In [5]:
# Extract important details from the file
import os
%load_ext autoreload
%autoreload 2
import pandas as pd
import pyspark
import json
import pyarrow
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from py4j.protocol import Py4JJavaError

from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, BooleanType

os.environ['SPARK_HOME'] = 'C:/Users/saul2/Spark_DF/spark-3.5.5-bin-hadoop3'
os.environ['HADOOP_HOME'] = 'C:/Users/saul2/Spark_DF/spark-3.5.5-bin-hadoop3'
os.environ['JAVA_HOME'] = 'C:/Program Files/Java/jdk1.8.0_202'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
spark = SparkSession.builder.appName("Extract_Gaming_Session").getOrCreate()

In [581]:
try:
    df = spark.read.parquet("C:/Users/saul2/OneDrive/Desktop/PastProjects/ETL_Project/VG_Sales/clean_data/clean_vgsales.parquet")
except AnalysisException as e:
    print(e)

In [717]:
directory = 'results'
if not os.path.exists(directory):
    os.makedirs(directory)

In [585]:
df = df.withColumn('Year', col('Year').cast(IntegerType()))

In [587]:
df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



In [591]:
#df.show()
#df.createOrReplaceTempView("VG_SALES")

In [727]:
# Get the top global sale game per platform
query = '''
    WITH GAME_GLOBAL_SALES AS (
    SELECT PLATFORM, MAX(GLOBAL_SALES) TOP_SELLER
    FROM VG_SALES
    GROUP BY PLATFORM
    )
    SELECT GGS.PLATFORM, GGS.TOP_SELLER, VG.NAME
    FROM GAME_GLOBAL_SALES GGS, VG_SALES VG
    WHERE GGS.PLATFORM = VG.PLATFORM AND GGS.TOP_SELLER = VG.GLOBAL_SALES
    ORDER BY TOP_SELLER
'''

query = '''
    WITH RANK_GAMES AS (
        SELECT PLATFORM, NAME, YEAR, GENRE, PUBLISHER, GLOBAL_SALES, RANK()OVER(PARTITION BY PLATFORM ORDER BY GLOBAL_SALES DESC) RANK
        FROM VG_SALES
    )
    SELECT PLATFORM, NAME, YEAR, GENRE, PUBLISHER, GLOBAL_SALES
    FROM RANK_GAMES
    WHERE RANK = 1
'''

result = spark.sql(query)
#result.show()

+--------+--------------------+----+------------+--------------------+------------+
|PLATFORM|                NAME|YEAR|       GENRE|           PUBLISHER|GLOBAL_SALES|
+--------+--------------------+----+------------+--------------------+------------+
|    2600|             Pac-Man|1982|      Puzzle|               Atari|        7.81|
|     3DO|         Policenauts|1995|   Adventure|Konami Digital En...|        0.06|
|     3DS| Pokemon X/Pokemon Y|2013|Role-Playing|            Nintendo|       14.35|
|      DC|     Sonic Adventure|1998|    Platform|                Sega|        2.42|
|      DS|New Super Mario B...|2006|    Platform|            Nintendo|       30.01|
|      GB|Pokemon Red/Pokem...|1996|Role-Playing|            Nintendo|       31.37|
|     GBA|Pokemon Ruby/Poke...|2002|Role-Playing|            Nintendo|       15.85|
|      GC|Super Smash Bros....|2001|    Fighting|            Nintendo|        7.07|
|     GEN|Sonic the Hedgehog 2|1992|    Platform|                Sega|      

In [751]:
# Find Platforms with At Least 5 Games in Top 100 Global Sales
query = '''
        WITH TOP_100 AS (
            SELECT PLATFORM, NAME, ROW_NUMBER()OVER(ORDER BY GLOBAL_SALES DESC) RANK
            FROM VG_SALES
        ), PLATFORM_TOP_5 AS (
            SELECT PLATFORM, COUNT(NAME) NO_GAMES
            FROM TOP_100
            WHERE RANK < 101
            GROUP BY PLATFORM
        )
        SELECT PLATFORM, NO_GAMES 
        FROM PLATFORM_TOP_5
        WHERE NO_GAMES > 4
        ORDER BY NO_GAMES 
    '''
result = spark.sql(query)
result.show()

+--------+--------+
|PLATFORM|NO_GAMES|
+--------+--------+
|     PS4|       5|
|      PS|       5|
|      GB|       6|
|     PS2|       6|
|     3DS|       7|
|     PS3|       9|
|      DS|      13|
|     Wii|      15|
|    X360|      16|
+--------+--------+



In [757]:
# 3. Normalize Sales Across Regions
# What were the percentage of each sales to the global sale?
# NA_SALES / GLOBAL_SALES
query = '''
        SELECT NAME, PLATFORM, YEAR, GENRE, PUBLISHER, ROUND(100 * (NA_SALES / GLOBAL_SALES),2) NA_PERCENT,
            ROUND(100 * (EU_Sales / GLOBAL_SALES),2) EU_PERCENT,
            ROUND(100 * (JP_Sales / GLOBAL_SALES),2) JP_PERCENT,
            ROUND(100 * (Other_Sales / GLOBAL_SALES),2) OTHER_PERCENT
        FROM VG_SALES
    '''
result = spark.sql(query)
#result.show()

+--------------------+--------+----+------------+--------------------+----------+----------+----------+-------------+
|                NAME|PLATFORM|YEAR|       GENRE|           PUBLISHER|NA_PERCENT|EU_PERCENT|JP_PERCENT|OTHER_PERCENT|
+--------------------+--------+----+------------+--------------------+----------+----------+----------+-------------+
|          Wii Sports|     Wii|2006|      Sports|            Nintendo|     50.15|     35.07|      4.56|        10.22|
|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|     72.27|       8.9|     16.92|         1.91|
|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|     44.25|     35.96|     10.58|         9.24|
|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|     47.73|     33.36|      9.94|         8.97|
|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|     35.93|     28.34|     32.58|         3.19|
|              Tetris|      GB|1989|      Puzzle|       

In [767]:
# 4. Find Games That Were Outliers in One Region
# Find games where the EU_Sales were in the top 5% but Global Sales were not.
query = '''
        WITH TOP_EU_SALES(
            SELECT NAME, PLATFORM, YEAR, GENRE, PUBLISHER, EU_SALES, ROW_NUMBER()OVER(ORDER BY EU_SALES DESC) EU_RANK, 
            GLOBAL_SALES, ROW_NUMBER()OVER(ORDER BY GLOBAL_SALES DESC) GLOBAL_RANK
            FROM VG_SALES
        ), TOP_5_COUNT(
            SELECT FLOOR(COUNT(NAME) *.05) TOP_5_PERCENT
            FROM VG_SALES
        )
        SELECT YEAR, NAME, PLATFORM, GENRE, PUBLISHER, EU_SALES, GLOBAL_SALES
        FROM TOP_EU_SALES, TOP_5_COUNT
        WHERE EU_RANK <= TOP_5_PERCENT AND GLOBAL_RANK > TOP_5_PERCENT
            AND YEAR IS NOT NULL
        ORDER BY YEAR
        
    '''
result = spark.sql(query)
result.show(truncate = False)

+----+------------------------------------------+--------+------------+----------------------------+--------+------------+
|YEAR|NAME                                      |PLATFORM|GENRE       |PUBLISHER                   |EU_SALES|GLOBAL_SALES|
+----+------------------------------------------+--------+------------+----------------------------+--------+------------+
|1992|The 7th Guest                             |PC      |Adventure   |Virgin Interactive          |0.77    |0.78        |
|1994|Star Wars: Dark Forces                    |PC      |Shooter     |LucasArts                   |0.77    |1.95        |
|1996|Diablo                                    |PC      |Role-Playing|Activision                  |1.58    |1.59        |
|1997|Tenchu: Stealth Assassins                 |PS      |Action      |Activision                  |0.64    |1.99        |
|1997|Time Crisis                               |PS      |Shooter     |Namco Bandai Games          |0.87    |1.68        |
|1998|FIFA 2000 

In [769]:
# 5. Yearly Top Genre per Region
# For each year and each region (e.g., NA_Sales), find the top-performing genre
# You learned how to pivot a column. Create a region using UNION ALL
query = '''
        WITH TOP_GENRE_SALE_PER_REGION AS (
            SELECT YEAR, GENRE, 'NA' AS REGION, ROUND(SUM(NA_SALES),4) SALES 
            FROM VG_SALES
            WHERE YEAR IS NOT NULL
            GROUP BY YEAR, GENRE
            UNION ALL 
            SELECT YEAR, GENRE, 'EU' AS REGION, ROUND(SUM(EU_SALES),4) SALES 
            FROM VG_SALES
            WHERE YEAR IS NOT NULL
            GROUP BY YEAR, GENRE
            UNION ALL 
            SELECT YEAR, GENRE, 'JP' AS REGION, ROUND(SUM(JP_SALES),4) SALES 
            FROM VG_SALES
            WHERE YEAR IS NOT NULL
            GROUP BY YEAR, GENRE
            UNION ALL 
            SELECT YEAR, GENRE, 'OTHER' AS REGION, ROUND(SUM(OTHER_SALES),4) SALES 
            FROM VG_SALES
            WHERE YEAR IS NOT NULL
            GROUP BY YEAR, GENRE
        ), RANK_GENRES (
            SELECT YEAR, GENRE, REGION, SALES, RANK()OVER(PARTITION BY YEAR, REGION ORDER BY SALES DESC) RANK_SALES
            FROM TOP_GENRE_SALE_PER_REGION
        ), FILTER_TOP_GENRE (
            SELECT YEAR, GENRE, REGION, SALES, RANK_SALES
            FROM RANK_GENRES
            WHERE RANK_SALES = 1
        ) 
        SELECT YEAR, 
            MAX(CASE WHEN REGION = "NA" AND SALES = 0 THEN 'No_Genre'
                     WHEN REGION = "NA" THEN GENRE END) AS TOP_NA_GENRE,
            MAX(CASE WHEN REGION = "EU" AND SALES = 0 THEN 'No_Genre'
                     WHEN REGION = "EU" THEN GENRE END) AS TOP_EU_GENRE,
            MAX(CASE WHEN REGION = "JP" AND SALES = 0 THEN 'No_Genre'
                     WHEN REGION = "JP" THEN GENRE END ) AS TOP_REGION_GENRE,
            MAX(CASE WHEN REGION = "OTHER" AND SALES = 0 THEN 'No_Genre'
                     WHEN REGION = "OTHER" THEN GENRE END) AS TOP_OTHER_GENRE
        FROM FILTER_TOP_GENRE
        GROUP BY YEAR
        ORDER BY YEAR
    '''

result = spark.sql(query)
#result.show(30)

In [775]:
# 6. Convert Rankings to Buckets
# Convert Global Sales ranks into buckets like:
# Rank 1–50 → Tier 1
# Rank 51–200 → Tier 2
# Rank > 200 → Tier 3

query = '''
        WITH RANK_GLOBAL_SALES (
            SELECT NAME, PLATFORM, YEAR, GENRE, PUBLISHER, GLOBAL_SALES, RANK()OVER(ORDER BY GLOBAL_SALES DESC) RANK_SALES
            FROM VG_SALES
        )
        SELECT NAME, PLATFORM, YEAR, GENRE, PUBLISHER, GLOBAL_SALES, 
            CASE WHEN RANK_SALES < 51 THEN '1'
                 WHEN RANK_SALES < 201 THEN '2'
                 ELSE '3'
            END AS SALES_TIER
        FROM RANK_GLOBAL_SALES
    '''

result = spark.sql(query)
#result.show(52)

In [777]:
# 7 Find Publishers That Had a Hot Streak
# Find publishers that had at least 3 consecutive years with global sales over 50M
year_interval = '''
        SELECT MIN(YEAR), MAX(YEAR)
        FROM VG_SALES
    '''

year_result = spark.sql(year_interval)
year_table = year_result.collect()[0]

years_df = spark.range(int(year_table[0]), int(year_table[1]) + 1).withColumnRenamed("id", "YEAR")
#years_df.show()
years_df.createOrReplaceTempView("Year_Table")

publisher_df = spark.sql("SELECT DISTINCT PUBLISHER FROM VG_SALES WHERE PUBLISHER IS NOT NULL")
publisher_df.createOrReplaceTempView("Publisher_Table")

Publisher_Year_DF = spark.sql('''
            SELECT y.YEAR, p.PUBLISHER
            FROM YEAR_TABLE y
            CROSS JOIN
            Publisher_Table p
        ''')

Publisher_Year_DF.createOrReplaceTempView("Year_Publisher_Table")

query = '''
        WITH YEARLY_SALES AS (
            SELECT YEAR, PUBLISHER, ROUND(SUM(GLOBAL_SALES),2) YEARLY_GLOBAL_SALES
            FROM VG_SALES
            WHERE YEAR IS NOT NULL
            GROUP BY YEAR, PUBLISHER
        ), INNER_JOIN_YEARS AS (
            SELECT YB.YEAR, YB.PUBLISHER, COALESCE(YS.YEARLY_GLOBAL_SALES,0) YEARLY_GLOBAL_SALES
            FROM YEARLY_SALES YS
            RIGHT JOIN Year_Publisher_Table YB
                ON YS.YEAR = YB.YEAR AND YS.PUBLISHER = YB.PUBLISHER
        ), GET_SALES AS (
            SELECT YEAR, PUBLISHER, YEARLY_GLOBAL_SALES 1_YEAR_SALE, 
                LEAD(YEARLY_GLOBAL_SALES,1,0)OVER(PARTITION BY PUBLISHER ORDER BY YEAR) 2_YEAR_SALE,
                LEAD(YEARLY_GLOBAL_SALES,2,0)OVER(PARTITION BY PUBLISHER ORDER BY YEAR) 3_YEAR_SALE
            FROM INNER_JOIN_YEARS
            ORDER BY PUBLISHER, YEAR
        )
        SELECT DISTINCT PUBLISHER, YEAR
        FROM GET_SALES
        WHERE 1_YEAR_SALE >= 50 AND 2_YEAR_SALE >= 50 AND 3_YEAR_SALE >= 50
    '''
result = spark.sql(query)
#result.show(100, truncate = False)

In [779]:
# 8. Genre Trends Over Time
query = '''
        WITH GENRE_YEARLY_SALES AS (
            SELECT YEAR, GENRE, ROUND(SUM(GLOBAL_SALES),2) CURR_SALES
            FROM VG_SALES
            WHERE YEAR IS NOT NULL
            GROUP BY YEAR, GENRE
        ), GENRE_PREV_SALES AS (
            SELECT YEAR, GENRE, CURR_SALES, 
                LAG(CURR_SALES, 1, 0) OVER(PARTITION BY GENRE ORDER BY YEAR) PREV_GLOBAL_SALES
            FROM GENRE_YEARLY_SALES
        )
        SELECT YEAR, GENRE, CURR_SALES, PREV_GLOBAL_SALES,
            ROUND(
                CASE 
                    WHEN PREV_GLOBAL_SALES = 0 THEN NULL
                    ELSE ((CURR_SALES - PREV_GLOBAL_SALES) / PREV_GLOBAL_SALES) * 100
                END
            ,2) PERCENTAGE_CHANGE
        FROM GENRE_PREV_SALES
        
    '''
result = spark.sql(query)
#result.show()

data = result.toPandas()
path = os.path.join(directory, 'YEARLY_GENRE_TREND.parquet')
data.to_parquet(path, engine='pyarrow') 
path = os.path.join(directory, 'YEARLY_GENRE_TREND.csv')
data.to_csv(path, index=False)
path = os.path.join(directory, 'YEARLY_GENRE_TREND.json')
data.to_json(path, orient='records', lines=True)

In [419]:
query = '''
        SELECT *
        FROM VG_SALES
        WHERE YEAR IS NOT NULL
        ORDER BY YEAR
    '''
result = spark.sql(query)
#result.show(11)

In [ ]:
# Now save them to the result file